In [10]:
import numpy as np

# Algebra de MCO: continuación del problema 2 (HMW 1)

## 2.1. Leverage Values

### 1.Obtenga el valor $h_{i i}$ para cada una de las observaciones $h_{i i}=X_i^{\prime}\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1} X_i$

In [11]:
##definimos matrices X y Y dadas por el ejercicio
y=np.array([[3],[1],[8],[3],[5]])

x=np.array([[1, 3, 5],
            [1, 1, 4],
            [1, 5, 6],
            [1, 2, 4],
            [1, 4, 6]])

In [12]:
hii = [ ]
for i in range(0,5):
    h_ii = x[i,:] @ np.round(np.linalg.inv(x.T @ x),4) @ x[i,:].T
    hii.append(h_ii)

hii = np.array(hii)
print(hii)

[0.2 0.7 0.7 0.7 0.7]


### 2.Muestre que: i) $0 \leq h_{i i} \leq 1$

In [13]:
print(np.all((hii >= 0) & (hii <= 1)))

True


ii) $h_{i i} \geq 1 / n$ si $X$ incluye un intercepto

In [14]:
print(np.all(hii >= 1 / len(x)))

True


iii) $\sum_{i=1}^n h_{i i}=k$

In [15]:
print(np.round(np.sum(hii),4) == len(x.T))

True


### 3.Para cada una de las observaciones, obtenga el estimador LOO:
$$
\hat{\beta}_{(-i)}=\left(\sum_{j \neq i} X_j X_j^{\prime}\right)^{-1}\left(\sum_{j \neq i} X_j Y_j\right)
$$

Note que por cada observación que saque, usted obtendrá los $\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}_2$. En total tendrá 15 estimaciones!

In [19]:
betahat_minusi = []
for i in range(len(x)):
    X_minus_i = np.delete(x, i, axis=0)
    y_minus_i = np.delete(y, i, axis=0)
    beta = np.round(np.linalg.inv(X_minus_i.T @ X_minus_i) @ X_minus_i.T @ y_minus_i,4)
    betahat_minusi.append(beta)

betahat_minusi = np.round(betahat_minusi,3)
print(betahat_minusi)

[[[ 4.25 ]
  [ 2.5  ]
  [-1.5  ]]

 [[ 5.333]
  [ 3.333]
  [-2.333]]

 [[ 2.   ]
  [ 1.667]
  [-0.667]]

 [[ 4.   ]
  [ 2.5  ]
  [-1.5  ]]

 [[ 4.   ]
  [ 2.5  ]
  [-1.5  ]]]


### 4. Para cada observación obtenga: $\tilde{Y}_i=X_i^{\prime} \hat{\beta}_{(-i)}$. Muestre los valores.

In [33]:
y_tilde = [np.round(x[i,:] @ betahat_minusi[i],4) for i in range(len(x))]
for i  in y_tilde:
    print(i)

[4.25]
[-0.666]
[6.333]
[3.]
[5.]


### 5.Para cada observación obtenga: $\tilde{e}_i=Y_i-\tilde{Y}_i$. Muestre los valores.

In [37]:
e_tilde = y - y_tilde
e_tilde = np.round(e_tilde,4)
print(e_tilde)

[[-1.25 ]
 [ 1.666]
 [ 1.667]
 [ 0.   ]
 [ 0.   ]]


### 6.Muestre que $\widetilde{e}_i=\left(1-h_{i i}\right)^{-1} \widehat{e_i}$. Muestre que los valores. conciden con aquellos del literal 5 .

In [41]:
beta_hat = np.linalg.inv(x.T @ x) @ x.T @ y
e_hat = np.round(y - x @ beta_hat,2)

e_tilde_i = e_hat.T / (1 - hii) 
e_tilde_i = np.round(e_tilde_i,2)
print(e_tilde_i)

print('Los valores coinciden')


[[-1.25  1.67  1.67 -0.   -0.  ]]
Los valores coinciden


## Matrices de Varianza y Covarianza

### 7.Obtenga el estimador de la varianza de lo erroes corregida por sesgo: $s^2=\frac{1}{n-k} \sum_{i=1}^n \hat{e}_i^2$. Muestre los valores.

In [43]:
n = 5
k = 2
s2 = (1 / (n - k)) * np.sum(e_hat**2)
print("S^2: ",s2)

S^2:  0.5


### 8.Obtenga el estimador de la varianza de los errores estandarizados: $\bar{\sigma}^2=\frac{1}{n} \sum_{i=1}^n\left(1-h_{i i}\right)^{-1} \hat{e}_i^2$. Muestre los valores.

In [53]:
sigma_bar2 = (1 / n) * np.sum((1 - h_ii)**-1 * e_hat**2)
print("Estimador de varianza de los errores estandarizados: " , (sigma_bar2))


Estimador de varianza de los errores estandarizados:  1.0000000000000095


### 9.Obtenga la matriz de varianza y covarianza bajo homocedasticidad $\tilde{\boldsymbol{V}}_\beta^Q=\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1} s^2$. Muestre los valores.

In [52]:
v_beta= np.linalg.inv(x.T @ x) * s2
print("Matriz de varianza y covarianza bajo homocedasticidad:\n", v_beta)

Matriz de varianza y covarianza bajo homocedasticidad:
 [[13.35  2.25 -4.  ]
 [ 2.25  0.5  -0.75]
 [-4.   -0.75  1.25]]


### 10. Obtenga el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC0: $\widehat{\boldsymbol{V}}_\beta^{\mathrm{HC}}=\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}\left(\sum_{i=1}^n X_i X_i^{\prime} \hat{e}_i^2\right)\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}$. Muestre los valores.

In [54]:
e_hat_flat = e_hat.flatten()
v_HC0 = np.linalg.inv(x.T @ x) @ np.sum([e_hat_flat[i]**2 * np.outer(x[i, :], x[i, :]) for i in range(n)], axis=0) @ np.linalg.inv(x.T @ x)
print("el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC0:\n", v_HC0)

el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC0:
 [[ 0.56   0.25  -0.25 ]
 [ 0.25   0.125 -0.125]
 [-0.25  -0.125  0.125]]


### 11. Obtenga el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad $\mathrm{HC} 1 \hat{\boldsymbol{V}}_{\boldsymbol{\beta}}^{\mathrm{HCl}}=\left(\frac{n}{n-k}\right)\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}\left(\sum_{i=1}^n X_i X_i^{\prime} \hat{e}_i^2\right)\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}$. Muestre los valores.

In [55]:
v_HC1 = (n / (n - k)) * v_HC0
v_HC1 = np.round(v_HC1,4)
print("l estimador de la matriz de varianza y convarianza consistente para heterocedasticidad", v_HC1)

l estimador de la matriz de varianza y convarianza consistente para heterocedasticidad [[ 0.9333  0.4167 -0.4167]
 [ 0.4167  0.2083 -0.2083]
 [-0.4167 -0.2083  0.2083]]


### 12.Obtenga el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC2 $\hat{\boldsymbol{V}}_{\boldsymbol{\beta}}^{\mathrm{HC} 2}=\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}\left(\sum_{i=1}^n\left(1-h_{i i}\right)^{-1} X_i X_i^{\prime} \hat{e}_i^2\right)\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}$. Muestre los valores.

In [56]:
v_HC2 = np.linalg.inv(x.T @ x) @ np.sum([(e_hat_flat[i]**2) * (1 - hii[i])**-1 * np.outer(x[i, :], x[i, :]) for i in range(n)
], axis=0) @ np.linalg.inv(x.T @ x)
v_HC2 = np.round(v_HC2, 4)
print("el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC2:\n", v_HC2)

el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC2:
 [[ 1.7833  0.8333 -0.8333]
 [ 0.8333  0.4167 -0.4167]
 [-0.8333 -0.4167  0.4167]]


### 13.Obtenga el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC3 $\hat{\boldsymbol{V}}_{\boldsymbol{\beta}}^{\mathrm{HC} 3}=\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}\left(\sum_{i=1}^n\left(1-h_{i i}\right)^{-2} X_i X_i^{\prime} \hat{e}_i^2\right)\left(\boldsymbol{X}^{\prime} \boldsymbol{X}\right)^{-1}$. Muestre los valores.

In [57]:
v_HC3 = np.linalg.inv(x.T @ x) @ np.sum([e_hat_flat[i]**2 * (1 - hii[i])**-2 * np.outer(x[i, :], x[i, :]) for i in range(n)
], axis=0) @ np.linalg.inv(x.T @ x)
v_HC3 = np.round(v_HC3, 4)
print("el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC3:\n", v_HC3)

el estimador de la matriz de varianza y convarianza consistente para heterocedasticidad HC3:
 [[ 5.8403  2.7778 -2.7778]
 [ 2.7778  1.3889 -1.3889]
 [-2.7778 -1.3889  1.3889]]
